In [10]:
#Checking wheather GPU is working or not
!nvidia-smi


Fri Dec 19 10:31:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.172.08             Driver Version: 570.172.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [11]:
# installing dataset and transformer
!pip install datasets transformers[sentencepiece] sacrebleu -q

In [12]:
# to remove version conflict of Protobuf so, downgrade version of Protobuf
!pip install protobuf==3.20.3 

In [13]:
!pip uninstall -y pyarrow datasets
!pip cache purge

# install fresh, modern, compatible versions
!pip install --no-cache-dir "datasets>=2.21.0" "pyarrow>=15.0.0"
!pip uninstall -y numpy
!pip install numpy==1.26.4

Found existing installation: pyarrow 22.0.0
Uninstalling pyarrow-22.0.0:
  Successfully uninstalled pyarrow-22.0.0
Found existing installation: datasets 4.4.1
Uninstalling datasets-4.4.1:
  Successfully uninstalled datasets-4.4.1
Files removed: 6
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 12.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 277.0 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<

In [14]:
# Importing all required modules
import os
import sys
import transformers
import torch  # pytorch Import
import sacrebleu
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset # for loading the dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM # For getting Embedding
from transformers import DataCollatorForSeq2Seq #getting sequential model and collator for loading batchwise of data
from torch.optim import AdamW # Optimizer



In [15]:
# Enter Access Token and rerun
from huggingface_hub import login
login(new_session=False)

# Note:
* I was using the free version of Kaggle, and the memory limit was getting exhausted while training the 1B-parameter model. Because of this constraint, I switched to using the 200M-parameter model instead.

In [16]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "ai4bharat/indictrans2-indic-en-1B"
LORA_REPO  = "Vir123-dev/indictrans2_hi_en_finetune_1B"

# Load tokenizer ONLY from base model
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True
)

# Attach LoRA adapters
model = PeftModel.from_pretrained(
    base_model,
    LORA_REPO
)

model = model.to("cuda")
model.eval()


tokenizer_config.json:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

tokenization_indictrans.py:   0%|          | 0.00/8.04k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-indic-en-1B:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


dict.SRC.json:   0%|          | 0.00/3.39M [00:00<?, ?B/s]

dict.TGT.json:   0%|          | 0.00/645k [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/869 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['target_parameters'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): IndicTransForConditionalGeneration(
      (model): IndicTransModel(
        (encoder): IndicTransEncoder(
          (embed_tokens): Embedding(122706, 1024, padding_idx=1)
          (embed_positions): IndicTransSinusoidalPositionalEmbedding()
          (layers): ModuleList(
            (0-17): 18 x IndicTransEncoderLayer(
              (self_attn): IndicTransAttention(
                (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=1024, out_features=4, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_featur

# The Dataset¶

* Source: https://huggingface.co/datasets/atrisaxena/mini-iitb-english-hindi

In [17]:
raw_dataset = load_dataset("atrisaxena/mini-iitb-english-hindi")

README.md:   0%|          | 0.00/193 [00:00<?, ?B/s]

data/train.parquet:   0%|          | 0.00/2.87M [00:00<?, ?B/s]

data/validation.parquet:   0%|          | 0.00/84.7k [00:00<?, ?B/s]

data/test.parquet:   0%|          | 0.00/500k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [18]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

# Observation for Statistics related to dataset

In [19]:
# Import required libraries
import numpy as np
import math
import nltk
nltk.download("punkt")  # one-time

def add_stats(example):
    text = example["translation"]["hi"]
    # guard
    if text is None: text = ""
    text = text.strip() # Removes unwanted spacing
    words = text.split()
    # sentence count (approx)
    sents = nltk.tokenize.sent_tokenize(text) if text else []
    example["num_words"] = len(words)
    example["num_chars"] = len(text)
    example["num_sentences"] = len(sents)
    return example

raw_dataset = raw_dataset.map(add_stats, batched=False)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/520 [00:00<?, ? examples/s]

Map:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [20]:
# Obtaining the Statistics:

def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))


=== TRAIN ===
Words: {'count': 20000, 'min': 0, 'p1': 1, 'p10': 1, 'median': 9.0, 'mean': 13.99495, 'std': 16.312517417538594, 'p90': 32, 'p99': 77, 'max': 330}
Chars: {'count': 20000, 'min': 0, 'p1': 3, 'p10': 7, 'median': 47.0, 'mean': 72.25215, 'std': 82.12149030782076, 'p90': 167, 'p99': 377, 'max': 1665}
Sentences: {'count': 20000, 'min': 0, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0619, 'std': 0.3868699910822756, 'p90': 1, 'p99': 3, 'max': 14}

=== VALIDATION ===
Words: {'count': 520, 'min': 3, 'p1': 5, 'p10': 8, 'median': 16.0, 'mean': 17.903846153846153, 'std': 8.824266400941944, 'p90': 29, 'p99': 47, 'max': 59}
Chars: {'count': 520, 'min': 19, 'p1': 25, 'p10': 39, 'median': 82.0, 'mean': 90.76923076923077, 'std': 45.64630551425374, 'p90': 150, 'p99': 235, 'max': 328}
Sentences: {'count': 520, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0057692307692307, 'std': 0.07573603333659698, 'p90': 1, 'p99': 1, 'max': 2}

=== TEST ===
Words: {'count': 2507, 'min': 3, 'p1': 

In [21]:
from datasets import DatasetDict

In [22]:
# Train has min length of sentences as 0 ('min': 0) so, we Remove these row from dataset
def not_empty(example):
    text = example["translation"]["hi"]
    return text is not None and len(text.strip()) > 0
    
clean_train = raw_dataset["train"].filter(not_empty)
clean_val   = raw_dataset["validation"].filter(not_empty)
clean_test  = raw_dataset["test"].filter(not_empty)

raw_dataset = DatasetDict({
    "train": clean_train,
    "validation": clean_val,
    "test": clean_test
})

Filter:   0%|          | 0/20000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/520 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [23]:
type(raw_dataset)

datasets.dataset_dict.DatasetDict

In [24]:
# compute p99 threshold('p99': 68 so, removing other outliers(longer than 68 words))
word_lengths = np.array(raw_dataset["train"]["num_words"])
p99_threshold = int(np.percentile(word_lengths, 99))
print("Removing sentences longer than:", p99_threshold, "words")
raw_dataset["train"] = raw_dataset["train"].filter(
    lambda ex: ex["num_words"] <= p99_threshold
)


Removing sentences longer than: 77 words


Filter:   0%|          | 0/19931 [00:00<?, ? examples/s]

In [25]:
# Obtaining the desired Statistics
def summary_stats(arr):
    arr = np.array(arr)
    return {
        "count": int(arr.size),
        "min": int(arr.min()) if arr.size>0 else None,
        "p1": int(np.percentile(arr, 1)) if arr.size>0 else None,
        "p10": int(np.percentile(arr, 10)) if arr.size>0 else None,
        "median": float(np.median(arr)) if arr.size>0 else None,
        "mean": float(arr.mean()) if arr.size>0 else None,
        "std": float(arr.std(ddof=0)) if arr.size>0 else None,
        "p90": int(np.percentile(arr, 90)) if arr.size>0 else None,
        "p99": int(np.percentile(arr, 99)) if arr.size>0 else None,
        "max": int(arr.max()) if arr.size>0 else None,
    }

for split in raw_dataset:
    d = raw_dataset[split]
    print(f"\n=== {split.upper()} ===")
    print("Words:", summary_stats(d["num_words"]))
    print("Chars:", summary_stats(d["num_chars"]))
    print("Sentences:", summary_stats(d["num_sentences"]))



=== TRAIN ===
Words: {'count': 19741, 'min': 1, 'p1': 1, 'p10': 1, 'median': 9.0, 'mean': 13.16326427232663, 'std': 13.325732174991717, 'p90': 31, 'p99': 60, 'max': 77}
Chars: {'count': 19741, 'min': 1, 'p1': 3, 'p10': 7, 'median': 47.0, 'mean': 68.22405146649106, 'std': 68.10624171169167, 'p90': 161, 'p99': 305, 'max': 700}
Sentences: {'count': 19741, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0604326021984702, 'std': 0.34258586219205384, 'p90': 1, 'p99': 3, 'max': 12}

=== VALIDATION ===
Words: {'count': 520, 'min': 3, 'p1': 5, 'p10': 8, 'median': 16.0, 'mean': 17.903846153846153, 'std': 8.824266400941944, 'p90': 29, 'p99': 47, 'max': 59}
Chars: {'count': 520, 'min': 19, 'p1': 25, 'p10': 39, 'median': 82.0, 'mean': 90.76923076923077, 'std': 45.64630551425374, 'p90': 150, 'p99': 235, 'max': 328}
Sentences: {'count': 520, 'min': 1, 'p1': 1, 'p10': 1, 'median': 1.0, 'mean': 1.0057692307692307, 'std': 0.07573603333659698, 'p90': 1, 'p99': 1, 'max': 2}

=== TEST ===
Words: {'c

In [26]:
# Sample Example
raw_dataset['train'][0]

{'translation': {'en': 'The occupation of keeping bees.',
  'hi': 'मधुमक्खियों को पालने का कार्य। '},
 'num_words': 5,
 'num_chars': 30,
 'num_sentences': 1}

In [27]:
from datasets import DatasetDict

# New desired sizes
N_TRAIN = 2000
N_VAL   = 150
N_TEST  = 250

# Downsample using .select()
small_train = raw_dataset["train"].select(range(N_TRAIN))
small_val   = raw_dataset["validation"].select(range(N_VAL))
small_test  = raw_dataset["test"].select(range(N_TEST))

# Create a new DatasetDict
small_dataset = DatasetDict({
    "train": small_train,
    "validation": small_val,
    "test": small_test
})

small_dataset

DatasetDict({
    train: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 150
    })
    test: Dataset({
        features: ['translation', 'num_words', 'num_chars', 'num_sentences'],
        num_rows: 250
    })
})

In [28]:
raw_dataset = small_dataset  # As said in Task 2(for 2000 pair of sentence)

# Applying Tokenization:- How we obtain Embedding
* Attention Mask = Padding Mask x look Ahead Mask
* Input_ids = input_ids are tokenized text converted into numeric indices from tokenizer vocabulary.
* The model converts input_ids to embeddings internally through an embedding layer.

# Pipeline
* Text → Tokens → IDs → Embeddings → Transformer
* "I love India"
*      ↓              (tokenization)
* ["I","love","India"]
*      ↓              (vocab lookup)
* [34, 91, 2563]  ← input_ids
*      ↓
* [embedding vectors] ← actual embeddings used by model

In [29]:
# sample Example;
text = "Hello Myself Virendra. A final year student at NIT Surat."
tokenizer("eng_Latn hin_Deva " + text)

{'input_ids': [4, 8, 36512, 64671, 85811, 69070, 2511, 44808, 69955, 6379, 1631, 72759, 52424, 75501, 24652, 2865, 27899, 56327, 11031, 6379, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [30]:
# Tokenize the Target language(hindi - hin_Deva) - Example
with tokenizer.as_target_tokenizer():
    print(tokenizer("मधुमक्खियों को पालने का कार्य। "))

{'input_ids': [260, 13596, 25457, 21009, 13596, 9992, 8965, 26281, 12444, 14668, 18629, 16043, 260, 9992, 18629, 260, 16419, 6739, 15711, 12256, 11002, 260, 9992, 6739, 260, 9992, 6739, 8537, 8965, 14668, 26446, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [31]:

# simple inference (model already on GPU by device_map)
SRC_TAG = "hin_Deva"
TGT_TAG = "eng_Latn"
text = "My name is Virendra, I am Currently a AI/ML Researcher"

tagged = f"{SRC_TAG} {TGT_TAG} {text}"

inputs = tokenizer(tagged, return_tensors="pt").to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        use_cache=False   
    )

print(tokenizer.decode(out[0], skip_special_tokens=True))


My name is Virendra and I am currently an AI / ML Researcher.


# Evaluation metrics

# Finding  BLEU AND CHRF:

1. BLEU: BLEU checks how many n-grams from the candidate sentence also appear in the reference sentence.
2. CHRF: Instead of words, CHRF compares character n-grams.

In [32]:
model.device

device(type='cuda', index=0)

In [38]:
n_samples = 10

# determine model device safely
try:
    model_device = next(model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Model device:", model_device)

preds = []
refs = []

for i in range(min(n_samples, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # handle multiple possible row formats
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        # trans might be a dict or a string; handle both
        if isinstance(trans, dict):
            eng = trans.get("en") or trans.get("eng") or ""
            hi_ref = trans.get("hi") or trans.get("hin") or ""
        else:
            # sometimes translation is a string (rare) — treat as source
            eng = str(trans)
            hi_ref = ""
    elif isinstance(row, dict):
        # maybe keys are directly 'en' and 'hi'
        eng = row.get("en") or row.get("eng") or row.get("source") or ""
        hi_ref = row.get("hi") or row.get("hin") or row.get("target") or ""
    else:
        # fallback: row itself might be the translation dict-like
        try:
            eng = row["en"]
            hi_ref = row["hi"]
        except Exception:
            # last resort: stringify
            eng = str(row)
            hi_ref = ""

    eng = (eng or "").strip()
    hi_ref = (hi_ref or "").strip()
    refs.append(eng if eng else "")  # keep alignment

    # add required tags
    tagged = f"{SRC_TAG} {TGT_TAG} {eng}"

    # tokenize -> torch tensors -> move to model device
    tokenized = tokenizer(tagged, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = model.generate(**tokenized, max_length=128, num_beams=4, use_cache=False ,early_stopping=True)

    pred_text = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred_text)


Model device: cuda:0


In [39]:
#BLEU:
bleu = sacrebleu.corpus_bleu(preds, [refs])
#CHRF:
chrf = sacrebleu.corpus_chrf(preds, [refs])

print(f"\nEvaluated {len(preds)} samples")
print("BLEU:", bleu.score)
print("CHRF:", chrf.score)

# show a few examples
for i in range(min(5, len(preds))):
    print(f"\n=== SAMPLE {i+1} ===")
    print("SRC :", (raw_dataset["test"][i].get("translation", raw_dataset["test"][i]).get("hi")
                    if isinstance(raw_dataset["test"][i], dict) and "translation" in raw_dataset["test"][i]
                    else (raw_dataset["test"][i].get("hi") if isinstance(raw_dataset["test"][i], dict) else str(raw_dataset["test"][i]))))
    print("PRED:", preds[i])
    print("REF :", refs[i])



Evaluated 10 samples
BLEU: 85.62931939766057
CHRF: 95.52889131758788

=== SAMPLE 1 ===
SRC : आपकी कार में ब्लैक बॉक्स?
PRED: A black box in your car?
REF : A black box in your car?

=== SAMPLE 2 ===
SRC : जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।
PRED: As America's road planners struggle to find the cash to mend a crumbling highway system many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.
REF : As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.

=== SAMPLE 3 ===
SRC : यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड़

# Finding BERTScore:

* BERTScore: Uses BERT (or RoBERTa, or mBERT) embeddings to compare every token in candidate with every token in reference.

In [40]:
!pip install bert-score
from bert_score import score

P, R, F1 = score(preds, refs, lang="hi")
print("BERTScore F1:", F1.mean().item())

BERTScore F1: 0.982215404510498


# Finding BLEURT:
* BLEURT computes a similarity score using a fine-tuned BERT model that predicts human judgment of translation quality.

In [41]:
# Required Packages for Bleurt
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

# Calculating Bleurt
import evaluate
bleurt = evaluate.load("bleurt")
results = bleurt.compute(predictions=preds, references=refs)
print(results)

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-sefejws3
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-sefejws3
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.


INFO:tensorflow:Config file found, reading.


INFO:tensorflow:Config file found, reading.


...

INFO:tensorflow:Loading model.


INFO:tensorflow:Loading model.


INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


{'scores': [1.0147972106933594, 0.8338326215744019, 0.8078286647796631, 0.7512096762657166, 0.7934170961380005, 1.005467176437378, 0.5963764786720276, 0.6852377653121948, 0.8363962769508362, 0.7843672633171082]}


# Finding COMET
* COMET predicts a score that strongly correlates with human judgment.

In [44]:
# Required package for Comet
!pip install -q unbabel-comet
from comet import download_model, load_from_checkpoint


# choose model variable 
translation_model = globals().get("translation_model", None) or globals().get("model", None)
if translation_model is None:
    raise ValueError("No translation model found. Load your model into `model` or `translation_model` first.")

# device: try to get model device (handles DeviceMap too)
try:
    model_device = next(translation_model.parameters()).device
except StopIteration:
    model_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SRC_TAG = "eng_Latn"
TGT_TAG = "hin_Deva"

srcs = []
preds = []
refs = []

n = 10
for i in range(min(n, len(raw_dataset["test"]))):
    row = raw_dataset["test"][i]

    # robust extraction of source & reference
    if isinstance(row, dict) and "translation" in row:
        trans = row["translation"]
        if isinstance(trans, dict):
            ref = (trans.get("en") or trans.get("eng") or "").strip()
            src= (trans.get("hi") or trans.get("hin") or "").strip()
        else:
            ref= str(trans).strip()
            src= ""
    elif isinstance(row, dict):
        ref = (row.get("en") or row.get("eng") or row.get("source") or "").strip()
        src = (row.get("hi") or row.get("hin") or row.get("target") or "").strip()
    else:
        # fallback
        ref = str(row).strip()
        src = ""

    srcs.append(src)
    refs.append(ref)

    # add language tags required by IndicTrans2
    tagged = f"{SRC_TAG} {TGT_TAG} {src}"

    # tokenize -> PyTorch tensors -> move to model device
    tokenized = tokenizer(tagged,
                          return_tensors="pt",
                          truncation=True,
                          padding=True,
                          max_length=128)
    tokenized = {k: v.to(model_device) for k, v in tokenized.items()}

    # generate
    with torch.no_grad():
        out_ids = translation_model.generate(**tokenized, max_length=128, num_beams=4,use_cache=False , early_stopping=True)

    pred = tokenizer.decode(out_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True).strip()
    preds.append(pred)

# ---- COMET evaluation ----
model_path = download_model("Unbabel/wmt22-comet-da")
comet_model = load_from_checkpoint(model_path)

# prepare data list for COMET
data = [{"src": s, "mt": p, "ref": r} for s, p, r in zip(srcs, preds, refs)]
# note: comet_model.predict returns a dict; 'scores' contains numeric values
comet_out = comet_model.predict(data, batch_size=8)
comet_scores = comet_out["scores"] if isinstance(comet_out, dict) and "scores" in comet_out else comet_out

print("Samples evaluated:", len(preds))
print("Mean COMET score:", float(sum(comet_scores) / len(comet_scores)))

# quick side-by-side preview
for i in range(len(preds)):
    print(f"\n--- SAMPLE {i+1} ---")
    print("SRC :", srcs[i])
    print("PRED:", preds[i])
    print("REF :", refs[i])


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytor

Samples evaluated: 10
Mean COMET score: 0.8820236563682556

--- SAMPLE 1 ---
SRC : आपकी कार में ब्लैक बॉक्स?
PRED: The black box in your car
REF : A black box in your car?

--- SAMPLE 2 ---
SRC : जबकि अमेरिका के सड़क योजनाकार, ध्वस्त होते हुए हाईवे सिस्टम को सुधारने के लिए धन की कमी से जूझ रहे हैं, वहीं बहुत-से लोग इसका समाधान छोटे से ब्लैक बॉक्स में देख रहे हैं, जो आपकी कार के डैशबोर्ड पर सफ़ाई से फिट हो जाता है।
PRED: While American road planners are struggling to find funding to fix the crumbling highway system, many are looking for a solution in a small black box that fits snugly on your car's dashboard.
REF : As America's road planners struggle to find the cash to mend a crumbling highway system, many are beginning to see a solution in a little black box that fits neatly by the dashboard of your car.

--- SAMPLE 3 ---
SRC : यह डिवाइस, जो मोटर-चालक द्वारा वाहन चलाए गए प्रत्येक मील को ट्रैक करती है तथा उस सूचना को अधिकारियों को संचारित करती है, आजकल अमेरिका की प्रमुख सड़कों का वित्त

In [43]:
# Saving the Model and tokenizer
model.save_pretrained("pt_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/dict.SRC.json',
 'tokenizer/dict.TGT.json',
 'tokenizer/model.SRC',
 'tokenizer/model.TGT',
 'tokenizer/added_tokens.json')